In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import random
import multiprocessing

from itertools import product
from datetime import date
from path import Path

import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'

from mlbt.multiprocess import process_jobs
from mlbt.run_bt import run_bt
from mlbt.feature_eng import define_feature_configs
from mlbt.load_data import LENGTH_RANKING

DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
s_csv = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

# Code to execute the pipeline inside a multiprocessing.Pool
# This creates the payloads that "run_frontend_payloads.ipynb" can use to generate the
# frontend payloads (.json loaded by the frontend)
ranking = ["equity_index", ""]


symbol_groups = list(
    {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }.keys()
)

features = define_feature_configs()
print(len(features))
c_s = (len(features) // 32) + 1 
c_s = 1
print('c_s', c_s)
data_sim = {
    "start_date": [date(2000, 1, 1)],
    "end_date": [date(2021, 1, 1)],
    "symbol_groups": [symbol_groups],
#     "symbols": [['@NQ#C', '@TU#C']],
#     "symbols": [[x] for x in LENGTH_RANKING],
    "bar_type": ["dollar"],
    "binarize": ["fixed_horizon"],
#     "binarize_params": [6, 12],
#     "binarize_params": [6],
    "binarize_params": [6, 12, 50, 100, 200],
#     "binarize_params": [100, 200],
#     "binarize_params": [200],
#     "binarize_params": [6, 25, 50],
#     "binarize_params": [25],
#     "binarize_params": [100],
#     "binarize_params": [6],
#     "binarize_params": [800],
    "alpha": ["none"],
#     "alpha": ["ma-cross_100_1000"],
#     "features": [features[i:i+c_s] for i in range(0, len(features), c_s)],
    "features": [features],
    "classifier": ["random_forest", "xgboost", "lgbm", "dummy"],
#     "classifier": ["xgboost"],
#     "classifier": ["dummy"],
#     "classifier": ["lgbm"],
    "num_threads": [32], # how many parallel hyperopti 
    "n_jobs": [32], # how many parallel infer
#     "num_threads": [8], # how many parallel hyperopti 
#     "n_jobs": [8], # how many parallel infer
#     "num_threads": [1], # how many parallel hyperopti 
#     "num_threads": [2], # how many parallel hyperopti 
    "feat_imp_n_estimators": [1000],
    "feat_imp_cv": [10],
    "func": [run_bt],
#     "feature_calc_only": [True],
#     "feature_imp_only": [True],
#     "check_completed": [True],
#     "optimize_hypers": [False],
    "reuse_hypers": [False],
#     "load_from_disk": [False],
#     "skip_feature_imp": [True],
#     "pca_transform": [False],
#     "standard_scale": [False],
#     "hypers_n_iter": [10],
}

C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


33
c_s 1


In [3]:
datas = list(dict(zip(data_sim.keys(), x)) for x in product(*data_sim.values()))
datas
# for data in datas:
#     data['features'] = list(sorted(data['features'], key=lambda x:random.random()))


[{'start_date': datetime.date(2000, 1, 1),
  'end_date': datetime.date(2021, 1, 1),
  'symbol_groups': ['agriculture',
   'currency',
   'energy',
   'equity_index',
   'interest_rate',
   'metals'],
  'bar_type': 'dollar',
  'binarize': 'fixed_horizon',
  'binarize_params': 6,
  'alpha': 'none',
  'features': [{'name': 'log_ret'},
   {'name': 'close', 'symbol': 'VIX.XO'},
   {'name': 'ffd', 'd': 0.5},
   {'name': 'roll', 'window': 10},
   {'name': 'rollimp', 'window': 10},
   {'name': 'amihud', 'window': 10},
   {'name': 'kyle', 'window': 10},
   {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 25},
   {'name': 'rollimp', 'window': 25},
   {'name': 'amihud', 'window': 25},
   {'name': 'kyle', 'window': 25},
   {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 50},
   {'name': 'rollimp', 'window': 50},
   {'name': 'amihud', 'window': 50},
   {'name': 'kyle', 'window': 50},
   {'name': 'ewm_

In [4]:
len(datas)

20

In [5]:
# [x['features'][0] for x in datas]

In [6]:
num_threads = max(1, min(multiprocessing.cpu_count(), len(datas)) - 0)
num_threads
      

20

In [7]:
# res = process_jobs(datas, num_threads=num_threads)

In [8]:
for data in datas:
    run_bt(**data)

2020-02-25 15:25:38,714 config: {'DATA_DIR': Path('C:\\Users\\doda/Dropbox/algotrading/data'), 'F_PAYLOAD_DIR': Path('C:\\Users\\doda/pr/fincl/frontend/public/payloads'), 'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'data_freq': 'minutely', 'downsampling': 'cusum', 'vol_estimate': 100, 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'features': [{'name': 'log_ret'}, {'name': 'close', 'symbol': 'VIX.XO'}, {'name': 'ffd', 'd': 0.5}, {'name': 'roll', 'window': 10}, {'name': 'rollimp', 'window': 10}, {'name': 'amihud', 'window': 10}, {'name': 'kyle', 'window': 10}, {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 25}, {'name': 'rollimp', 'window': 25}, {'name': 'amihud', 'window': 25}, {'name': 'kyle', 'window': 25}, {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 50}, {'name': 'rollimp', 'window': 50}, {'name': '

2020-02-25 15:25:44,874 XG#C: Feature engineering for 33 features
2020-02-25 15:25:45,226 Joined 33 features into (71217, 33) shape
2020-02-25 15:25:45,227 EX#C: Feature engineering for 33 features
2020-02-25 15:25:45,578 Joined 33 features into (74018, 33) shape
2020-02-25 15:25:45,578 @YM#C: Feature engineering for 33 features
2020-02-25 15:25:45,911 Joined 33 features into (39311, 33) shape
2020-02-25 15:25:45,912 @BO#C: Feature engineering for 33 features
2020-02-25 15:25:46,273 Joined 33 features into (93715, 33) shape
2020-02-25 15:25:46,274 @C#C: Feature engineering for 33 features
2020-02-25 15:25:46,629 Joined 33 features into (80542, 33) shape
2020-02-25 15:25:46,630 @FV#C: Feature engineering for 33 features
2020-02-25 15:25:46,973 Joined 33 features into (53497, 33) shape
2020-02-25 15:25:46,973 @TY#C: Feature engineering for 33 features
2020-02-25 15:25:47,319 Joined 33 features into (55083, 33) shape
2020-02-25 15:25:47,319 @TU#C: Feature engineering for 33 features
2020-

2020-02-25 15:25:56,672 Dropped label 0.0 0.012008470905328007
2020-02-25 15:25:56,675 @SB#C: Get bins and feature imps
2020-02-25 15:25:56,693 Dropped label 0.0 0.01769161037517911
2020-02-25 15:25:56,696 GAS#C: Get bins and feature imps
2020-02-25 15:25:56,713 Dropped label 0.0 0.014445620934805626
2020-02-25 15:25:56,715 LF#C: Get bins and feature imps
2020-02-25 15:25:56,735 Dropped label 0.0 0.005299040356715887
2020-02-25 15:25:56,738 LG#C: Get bins and feature imps
2020-02-25 15:25:56,752 Dropped label 0.0 0.011036065863241072
2020-02-25 15:25:56,755 QCL#C: Get bins and feature imps
2020-02-25 15:25:56,774 Dropped label 0.0 0.004039408866995074
2020-02-25 15:25:56,777 QHO#C: Get bins and feature imps
2020-02-25 15:25:56,789 Dropped label 0.0 0.0008529304252467406
2020-02-25 15:25:56,792 QNG#C: Get bins and feature imps
2020-02-25 15:25:56,813 Dropped label 0.0 0.005031324049728957
2020-02-25 15:25:56,816 QPA#C: Get bins and feature imps
2020-02-25 15:25:56,831 Dropped label 0.0 

2020-02-25 15:35:02,053 Running validation for RandomForestClassifier: round 5/5
C:\Users\doda\pr\fincl\backend\mlbt\reporting.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  events["y_pred_proba"] = y_pred_proba
C:\Users\doda\pr\fincl\backend\mlbt\reporting.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  events["y_pred"] = y_pred
2020-02-25 15:35:33,933 Run finished: Feature Importance OOS Score: 0.5102297193610393
2020-02-25 15:35:33,934 F1 Score: 0.5071336796810123
2020-02-25 15:35:34,225 config: {'DATA_DIR': Path('C:\\Users\\d

2020-02-25 15:35:37,467 @HE#C: Feature engineering for 33 features
2020-02-25 15:35:37,780 Joined 33 features into (61176, 33) shape
2020-02-25 15:35:37,781 @EMD#C: Feature engineering for 33 features
2020-02-25 15:35:38,099 Joined 33 features into (67589, 33) shape
2020-02-25 15:35:38,099 @NKD#C: Feature engineering for 33 features
2020-02-25 15:35:38,425 Joined 33 features into (73050, 33) shape
2020-02-25 15:35:38,426 @NE#C: Feature engineering for 33 features
2020-02-25 15:35:38,727 Joined 33 features into (45459, 33) shape
2020-02-25 15:35:38,728 @RP#C: Feature engineering for 33 features
2020-02-25 15:35:39,030 Joined 33 features into (33820, 33) shape
2020-02-25 15:35:39,030 BD#C: Feature engineering for 33 features
2020-02-25 15:35:39,343 Joined 33 features into (68392, 33) shape
2020-02-25 15:35:39,343 EZ#C: Feature engineering for 33 features
2020-02-25 15:35:39,667 Joined 33 features into (110422, 33) shape
2020-02-25 15:35:39,668 XG#C: Feature engineering for 33 features
20

...

2020-02-25 17:12:42,867 Dropped label 0.0 0.002329070835106264
2020-02-25 17:12:42,870 @C#C: Get bins and feature imps
2020-02-25 17:12:42,889 Dropped label 0.0 0.004587533994451007
2020-02-25 17:12:42,892 @FV#C: Get bins and feature imps
2020-02-25 17:12:42,909 Dropped label 0.0 0.001930422639339549
2020-02-25 17:12:42,911 @TY#C: Get bins and feature imps
2020-02-25 17:12:42,927 Dropped label 0.0 0.002877122877122877
2020-02-25 17:12:42,930 @TU#C: Get bins and feature imps
2020-02-25 17:12:42,944 Dropped label 0.0 0.00615326059375793
2020-02-25 17:12:42,946 @UB#C: Get bins and feature imps
2020-02-25 17:12:42,959 Dropped label 0.0 0.002730464178910415
2020-02-25 17:12:42,961 @O#C: Get bins and feature imps
2020-02-25 17:12:42,977 Dropped label 0.0 0.0027116972759768272
2020-02-25 17:12:42,980 @SM#C: Get bins and feature imps
2020-02-25 17:12:42,995 Dropped label 0.0 0.0027520759193357057
2020-02-25 17:12:42,998 @S#C: Get bins and feature imps
2020-02-25 17:12:43,015 Dropped label 0.0 

2020-02-25 17:13:05,022 Getting model lgbm
2020-02-25 17:13:05,023 hyperparam search n_iter=25 for lgbm on num_threads=32 and n_jobs=-1
C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning
2020-02-25 17:13:35,579 Using hypers {'num_leaves': 62, 'n_estimators': 25, 'max_depth': 7, 'learning_rate': 0.02, 'colsample_bytree': 0.8999999999999999}
2020-02-25 17:13:35,580 Getting reports for LGBMClassifier
2020-02-25 17:13:35,742 Running validation for LGBMClassifier: round 1/5
2020-02-25 17:13:36,789 Running validation for LGBMClassifier: round 2/5
2020-02-25 17:13:38,083 Running validation for LGBMClassifier: round 3/5
2020-02-25 17:13:39,592 Running validation for LGBMClassifier: round 4/5
2020-02-25 17:13:41,286 Running validation for LGBMClassifier: round 5/5
C:\Users\doda\pr\fincl\backend\mlbt\reporting.py:77: SettingWithCopyW

2020-02-25 17:14:05,298 Joined 33 features into (92410, 33) shape
2020-02-25 17:14:05,299 @CD#C: Feature engineering for 33 features
2020-02-25 17:14:05,629 Joined 33 features into (86716, 33) shape
2020-02-25 17:14:05,629 @ED#C: Feature engineering for 33 features
2020-02-25 17:14:05,939 Joined 33 features into (48965, 33) shape
2020-02-25 17:14:05,939 @ES#C: Feature engineering for 33 features
2020-02-25 17:14:06,245 Joined 33 features into (58387, 33) shape
2020-02-25 17:14:06,246 @NQ#C: Feature engineering for 33 features
2020-02-25 17:14:06,540 Joined 33 features into (31156, 33) shape
2020-02-25 17:14:06,540 @GF#C: Feature engineering for 33 features
2020-02-25 17:14:06,837 Joined 33 features into (41686, 33) shape
2020-02-25 17:14:06,837 @LE#C: Feature engineering for 33 features
2020-02-25 17:14:07,144 Joined 33 features into (58349, 33) shape
2020-02-25 17:14:07,145 @HE#C: Feature engineering for 33 features
2020-02-25 17:14:07,450 Joined 33 features into (61176, 33) shape
202

2020-02-25 17:14:19,469 Dropped label 0.0 0.004587533994451007
2020-02-25 17:14:19,472 @FV#C: Get bins and feature imps
2020-02-25 17:14:19,488 Dropped label 0.0 0.001930422639339549
2020-02-25 17:14:19,491 @TY#C: Get bins and feature imps
2020-02-25 17:14:19,507 Dropped label 0.0 0.002877122877122877
2020-02-25 17:14:19,510 @TU#C: Get bins and feature imps
2020-02-25 17:14:19,525 Dropped label 0.0 0.00615326059375793
2020-02-25 17:14:19,527 @UB#C: Get bins and feature imps
2020-02-25 17:14:19,541 Dropped label 0.0 0.002730464178910415
2020-02-25 17:14:19,543 @O#C: Get bins and feature imps
2020-02-25 17:14:19,559 Dropped label 0.0 0.0027116972759768272
2020-02-25 17:14:19,562 @SM#C: Get bins and feature imps
2020-02-25 17:14:19,577 Dropped label 0.0 0.0027520759193357057
2020-02-25 17:14:19,580 @S#C: Get bins and feature imps
2020-02-25 17:14:19,599 Dropped label 0.0 0.002445699644791242
2020-02-25 17:14:19,601 @W#C: Get bins and feature imps
2020-02-25 17:14:19,624 Dropped label 0.0 

2020-02-25 17:14:41,593 Getting model dummy
2020-02-25 17:14:41,594 Using hypers {}
2020-02-25 17:14:41,595 Getting reports for DummyClassifier
2020-02-25 17:14:41,737 Running validation for DummyClassifier: round 1/5
C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
2020-02-25 17:14:42,272 Running validation for DummyClassifier: round 2/5
C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
2020-02-25 17:14:42,946 Running validation for DummyClassifier: round 3/5
C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", F

In [9]:
%debug

2020-02-25 17:15:06,254 No traceback has been produced, nothing to debug.
